In [28]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [29]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [30]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [31]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [32]:
scenarios = ["Sentence field skipped during question creation"]

In [33]:
prompt_uc = """
Use case name: Create questions for the selection test.
Actor": administrators.
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7:Admin presses one word on vocabulary word list.
Step 8:The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9:Admin presses button "Add new question".
Step 10:The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These compulsory fields are: text field "sentence" for inputting sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...'and "sentence" must not be skip,  a valid sentence must be below 250 characters and not contains any special character. one text field 'correct answer' for Admins to input the correct answer word, three text field 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills the field "sentence" with a valid sentence has a symbol '...', 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 14: Admin presses button 'Save' to save the question information.
Step 15: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow: 
Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.

Alternative flow 2: Admin skip compulsory field "sentence".
At step 13 of the basic flow: Admin field all the compulsory field except field "sentence".
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 3: Admin skip compulsory field "'correct answer'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'correct answer'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 4: Admin skip compulsory field 'other option 1'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 1'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 5: Admin skip compulsory field 'other option 2'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 2'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 6: Admin skip compulsory field 'other option 3'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 3'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 7: Admin choose cancel the question and choose to stay.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Stay'.
Step 16: The dialog closed and the work on that question was kept.
Step 17: Admin presses button 'Save'.

Alternative flow 8: Admin enters invalid sentences.
At step 13 of the basic flow: Admin fills all the other compulsory field but fills in the 'sentence' field by an invalid sentence.
Step 14: System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 16: Admin re-enter the missing field.
Step 17: Admin presses button 'Save'.

Exception flow: 
Exception flow 1: Admin leave by button back on browser without saving the question.
At step 13 of the basic flow: Admin presses the button back on the browser. 
Step 14: The system redirects Admin to the "Word Information" page of selected word and Question is not saved. Question is not saved.

Exception flow 2: Admin choose cancel the question and confirm the cancel.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Confirm'.
Step 16: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.

Pre Condition: User logged in as Admin.
Post condition: System save the question to the database, Question then can be view in the 'Word Inforamtion' page and can be edited or deleted by Admins and when Learner review this word, this question can be shown as a review test for learner.
"""

In [34]:
for scenario in scenarios:
        promptTestCase = [
                { "role": "system", "content": SYSTEM_PROMPT_8},
                { "role": "user", "content": scenario + "\n" + prompt_uc}
                ]
        gpt_response = ask(promptTestCase, client, model)
        json_data = json.loads(gpt_response)
        # write_json_to_txt_raw(json_data, scenario + "-Raw" + ".txt")
        print(gpt_response)
# promptSelfRef = [
#     { "role": "system", "content": SELF_REF},
#     { "role": "user", "content": scenarios[2] + "\n" + gpt_response}
# ]
# final_response = ask(promptSelfRef, client, model)
# json_fin = json.loads(final_response)
# write_json_to_txt(json_fin, scenarios[2] + ".txt")

{
  "Test Case 1": {
    "testCaseName": "Create a valid question for the selection test",
    "objective": "Verify that an admin can successfully create a question for the selection test with all required fields filled correctly",
    "testSteps": [
      "Step 1: Log in as Admin.",
      "Step 2: Press the tab 'Lesson Management' on the navbar.",
      "Step 3: Select a Lesson from the Lesson list.",
      "Step 4: Select a Topic from the Topic list of the selected Lesson.",
      "Step 5: Select a word from the vocabulary word list.",
      "Step 6: Press the button 'Add new question'.",
      "Step 7: Press option add 'Create questions for the selection test'.",
      "Step 8: Fill the 'sentence' field with a valid sentence including '...'.",
      "Step 9: Fill the 'correct answer' field with the correct word.",
      "Step 10: Fill 'other option 1', 'other option 2', 'other option 3' fields with different wrong answers.",
      "Step 11: Press the button 'Save'."
    ],
    "expe

In [35]:
for scenario in scenarios:
        promptTestCase = [
                { "role": "system", "content": SYSTEM_PROMPT_8},
                { "role": "user", "content": scenario + "\n" + prompt_uc}
                ]
        gpt_response = ask(promptTestCase, client, model)
        json_data = json.loads(gpt_response)
        # write_json_to_txt_raw(json_data, scenario + "-Raw" + ".txt")
        print(gpt_response)
# promptSelfRef = [
#     { "role": "system", "content": SELF_REF},
#     { "role": "user", "content": scenarios[2] + "\n" + gpt_response}
# ]
# final_response = ask(promptSelfRef, client, model)
# json_fin = json.loads(final_response)
# write_json_to_txt(json_fin, scenarios[2] + ".txt")

{
  "Test Case 1": {
    "testCaseName": "Create a valid question for the selection test",
    "objective": "Verify that the admin can successfully create a question for the selection test with all required fields filled correctly",
    "testSteps": [
      "Step 1: Log in as Admin.",
      "Step 2: Press the tab 'Lesson Management' on the navbar.",
      "Step 3: Select a Lesson from the Lesson list.",
      "Step 4: Select a Topic from the Topic list of the selected Lesson.",
      "Step 5: Select a word from the vocabulary word list.",
      "Step 6: Press the button 'Add new question'.",
      "Step 7: Press option add 'Create questions for the selection test'.",
      "Step 8: Fill the 'sentence' field with a valid sentence that includes '...'.",
      "Step 9: Fill the 'correct answer' field with the correct word.",
      "Step 10: Fill 'other option 1', 'other option 2', 'other option 3' fields with different wrong answers.",
      "Step 11: Press the button 'Save'."
    ],
    

In [36]:
for scenario in scenarios:
        promptTestCase = [
                { "role": "system", "content": SYSTEM_PROMPT_8},
                { "role": "user", "content": scenario + "\n" + prompt_uc}
                ]
        gpt_response = ask(promptTestCase, client, model)
        json_data = json.loads(gpt_response)
        # write_json_to_txt_raw(json_data, scenario + "-Raw" + ".txt")
        print(gpt_response)
# promptSelfRef = [
#     { "role": "system", "content": SELF_REF},
#     { "role": "user", "content": scenarios[2] + "\n" + gpt_response}
# ]
# final_response = ask(promptSelfRef, client, model)
# json_fin = json.loads(final_response)
# write_json_to_txt(json_fin, scenarios[2] + ".txt")

{
  "Test Case 1": {
    "testCaseName": "Create a valid question for the selection test",
    "objective": "Verify that an admin can successfully create a question for the selection test with all required fields filled correctly",
    "testSteps": [
      "Step 1: Log in as Admin.",
      "Step 2: Press the tab 'Lesson Management' on the navbar.",
      "Step 3: Select a Lesson from the Lesson list.",
      "Step 4: Select a Topic from the Topic list of the selected Lesson.",
      "Step 5: Select a word from the vocabulary word list.",
      "Step 6: Press the button 'Add new question'.",
      "Step 7: Press option add 'Create questions for the selection test'.",
      "Step 8: Fill the 'sentence' field with a valid sentence that includes '...'.",
      "Step 9: Fill the 'correct answer' field with the correct word.",
      "Step 10: Fill 'other option 1', 'other option 2', 'other option 3' fields with different wrong answers.",
      "Step 11: Press the button 'Save'."
    ],
    "